In [ ]:
import os
from dotenv import load_dotenv
from pprint import pprint

from stravalib import Client

load_dotenv(override=True)

In [ ]:
client = Client(
    access_token=os.getenv("STRAVA_ACCESS_TOKEN"),
    token_expires=int(os.getenv("STRAVA_TOKEN_EXPIRES")),
    refresh_token=os.getenv("STRAVA_REFRESH_TOKEN"),
)
athlete = client.get_athlete()

In [ ]:
# https://stravalib.readthedocs.io/en/latest/reference/api/stravalib.model.DetailedAthlete.html#stravalib.model.DetailedAthlete
print(athlete.id)
print(athlete.firstname)
print(athlete.lastname)
print(athlete.sex)
print(athlete.athlete_type)
print(athlete.follower_count)
print(athlete.bikes)
print(athlete.shoes)

In [ ]:
# athlete.model_dump()

In [ ]:
# https://stravalib.readthedocs.io/en/latest/reference/api/stravalib.model.AthleteStats.html#stravalib.model.AthleteStats
print(athlete.stats.all_run_totals)
print(athlete.stats.ytd_run_totals)
print(athlete.stats.recent_run_totals)
print(athlete.stats.biggest_ride_distance)

In [ ]:
athlete.stats.model_dump()

In [ ]:
activities = client.get_activities(limit=15)

In [ ]:
for activity in activities:
    print(f"{activity.name} | {activity.kudos_count} | {activity.distance} | {activity.average_heartrate}")

In [ ]:
for activity in activities:
    print(activity.model_dump())
    break

In [ ]:
import polars as pl

data = []
for activity in activities:
    data.append(activity.model_dump())

df = pl.DataFrame(data)
df

In [ ]:
df.get_column("sport_type").unique()

In [ ]:
df.columns

In [ ]:
for row in df.iter_rows():
    # print(row[22])
    print(row[26])

In [ ]:
df.get_column("map").first()

In [ ]:
import polyline

decoded_polyline = polyline.decode(df.get_column("map").first()["summary_polyline"])

In [ ]:
import ezgpx
gpx = ezgpx.GPX()
trk = ezgpx.gpx_elements.Track()
trk_seg = ezgpx.gpx_elements.TrackSegment()
for lat, lon in decoded_polyline:
    trk_seg.trkpt.append(ezgpx.gpx_elements.WayPoint(
        tag="trkpt", lat=lat, lon=lon))
trk.trkseg.append(trk_seg)
gpx.gpx.trk.append(trk)

# DEFAULT_PRECISION = 10
# DEFAULT_PRECISION_DICT = {
#     "lat_lon": DEFAULT_PRECISION,
#     "elevation": DEFAULT_PRECISION,
#     "distance": DEFAULT_PRECISION,
#     "duration": DEFAULT_PRECISION,
#     "speed": DEFAULT_PRECISION,
#     "rate": DEFAULT_PRECISION,
#     "default": DEFAULT_PRECISION,
# }
# DEFAULT_TIME_FORMAT = "%Y-%m-%dT%H:%M:%SZ"
# gpx._precisions = DEFAULT_PRECISION_DICT
# gpx._time_format = DEFAULT_TIME_FORMAT
# gpx._time_data = False
gpx._time_data = True
gpx._ele_data = True

In [ ]:
gpx.to_polars(["lat", "lon"])

In [ ]:
ezgpx.PlotlyPlotter(gpx).plot(color="rgb(255,0,0)")

In [ ]:
ezgpx.MatplotlibPlotter(gpx).plot()

In [ ]:
df_test = (
    df.select(["sport_type", "distance", "start_date_local"])
    # .filter(pl.col("sport_type").is_in(sport_types))
    # .with_columns(pl.col("start_date_local").str.to_datetime("%Y-%m-%dT%H:%M:%S+00:00"))
    .with_columns(
        pl.col("start_date_local").dt.iso_year().alias("iso_year"),
        pl.col("start_date_local").dt.week().alias("week"),
    )
    .with_columns(
        pl.concat_str([pl.col("iso_year"), pl.lit("-"), pl.col("week")]).alias(
            "year_week"
        )
    )
    .group_by(["year_week", "sport_type"])
    .agg(pl.col("distance").sum() / 1000)  # Convert to km
    .sort(["year_week"])
)
df_test

In [ ]:
min_date = df.select(pl.col("start_date_local").min()).item()
max_date = df.select(pl.col("start_date_local").max()).item()
print(min_date)
print(max_date)

In [ ]:
min_date = df.select(pl.col("start_date_local").min())
max_date = df.select(pl.col("start_date_local").max())

min_date

In [ ]:
min_monday = df.select(
    (
        pl.col("start_date_local").min()
        - pl.duration(days=pl.col("start_date_local").min().dt.weekday() - 1)
    ).alias("week_monday")
).item()

min_monday

In [ ]:
import datetime

now = datetime.datetime.now().date()
before = now - datetime.timedelta(30)
print(now, before)

In [ ]:
min_monday = df.select(
    (
        pl.col("start_date_local").min()
        - pl.duration(days=pl.col("start_date_local").min().dt.weekday() - 1)
    ).alias("week_monday")
).item()
max_monday = df.select(
    (
        pl.col("start_date_local").max()
        - pl.duration(days=pl.col("start_date_local").max().dt.weekday() - 1)
    ).alias("week_monday")
).item()

print(min_monday)
print(max_monday)

calendar = (
    pl.date_range(min_monday, max_monday, interval="1w", eager=True)
    .to_frame("date")
    .with_columns(
        pl.col("date").dt.iso_year().alias("iso_year"),
        pl.col("date").dt.week().alias("week"),
    )
    .with_columns(
        pl.format("{}-{}", pl.col("iso_year"), pl.col("week")).alias(
            "year_week"
        )
    )
    .select("year_week")
    .unique()
    .sort("year_week")
)
calendar

In [ ]:
pl.date_range(min_date, max_date, interval="1w", eager=True).to_frame("date")

In [ ]:
sports = pl.DataFrame({"sport_type": df_test.select("sport_type").unique().to_series().to_list()})
full_index = calendar.join(sports, how="cross")
full_index

In [ ]:
final = (
    full_index
    .join(df_test, on=["year_week", "sport_type"], how="left")
    .with_columns(pl.col("distance").fill_null(0))
    .sort(["year_week", "sport_type"])
)
final

In [ ]:
# client.get_athlete_stats()
# client.get_athlete_zones()

In [ ]:
for kom in client.get_athlete_koms(athlete.id, limit=3):
    pprint(kom.model_dump())

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_annotation(
    text="Absolutely-positioned annotation",
    xref="paper",
    yref="paper",
    x=0.5,
    y=0.5,
    textfont={"color": "#FF0000"},
    borderwidth=2,
    bordercolor="#FF0000",
    showarrow=False,
)
fig

ValueError: Received col parameter but not row.
row and col must be specified together